In [28]:
%load_ext autoreload
%autoreload 2

In [29]:
import numpy as np
import sys
sys.path.append('..')
from capacity_management.src.da import DeferredAcceptance
from capacity_management.src.mnl import MNL

In [18]:
n = 80 
q = 22
p = .1
t = 1/1000

In [16]:
q1hat = 1/(1-p) * (q - np.sqrt(q*p*(1-p)*2*np.log(3/t)))

In [17]:
q1hat

18.188068381431158

In [20]:
priorities = np.random.uniform(size=(n,3))

In [63]:
utilities = np.tile([6,5,4], (n,1))
mnl = MNL(utilities, num_students=n)
preferences = mnl.sample_preference_ordering()

In [72]:
da = DeferredAcceptance(preferences,priorities,np.array([22,22,22]))
assignment = da.da()
assignment

{0: array([ 7, 42, 10, 32, 45, 72, 48,  1, 53, 46,  5, 34, 68, 24, 78, 73, 55,
        23, 21, 33, 43,  6]),
 1: array([57, 36,  4, 14, 77, 60, 31, 15,  0, 19, 66, 28, 52, 39, 11, 69, 76,
        40, 18, 47,  3, 56]),
 2: array([26, 74, 59, 16, 49, 41, 63, 38, 27, 50, 29, 79, 22, 70,  8, 62, 12,
        17, 67,  2, 75, 44])}

In [157]:
def simulate_dropout(assignment):
    after_dropout = {}
    leaving_list = []
    for school, students in assignment.items():
        leaving = np.array(np.random.binomial(size=len(students), n=1, p=p), dtype=bool)
        after_dropout[school] = students[~leaving]
        leaving_list = np.append(leaving_list, students[leaving])
    return after_dropout, leaving_list
        

In [158]:
after_dropout, leaving = simulate_dropout(assignment)
after_dropout

{0: array([ 7, 32, 72, 48,  1, 53, 46, 68, 78, 73, 55, 23, 21, 43,  6]),
 1: array([57, 36,  4, 14, 77, 31, 15,  0, 66, 28, 52, 39, 11, 69, 76, 40, 18,
        47,  3, 56]),
 2: array([26, 74, 59, 16, 49, 41, 63, 27, 29, 22, 70,  8, 62, 12, 17, 67,  2,
        75, 44])}

In [159]:
leaving

array([42., 10., 45.,  5., 34., 24., 33., 60., 19., 38., 50., 79.])

In [160]:
def get_rank_of_school(students, school, preferences):
    return np.argwhere(preferences[students,:]==school)[:,1]

In [161]:
prefer_other_idx = np.argwhere(np.argwhere(preferences[after_dropout[0],:]==0)[:,1]-np.argwhere(preferences[after_dropout[0],:]==1)[:,1]>0).flatten()
prefer_other = after_dropout[0][prefer_other_idx]
preferences[prefer_other,:]

array([[2, 1, 0]])

In [162]:
def students_preferring_sch2_to_sch1(sch1_rank, sch2_rank, students):
#     sch1_rank = get_rank_of_school(students, sch1, preferences)
#     sch2_rank = get_rank_of_school(students, sch2, preferences)
    idx_prefer_other = np.argwhere(sch1_rank - sch2_rank > 0).flatten()
    return students[idx_prefer_other]

In [163]:
def identify_movers(after_dropout, preferences):
    movers = {}
    for sch1, students in after_dropout.items():
        sch1_rank = get_rank_of_school(students, sch1, preferences)
        movers[sch1] = {}
        for sch2 in after_dropout.keys():
            if sch2 == sch1:
                continue

            sch2_rank = get_rank_of_school(students, sch2, preferences)
            movers[sch1][sch2] = students_preferring_sch2_to_sch1(sch1_rank, sch2_rank, students)
    return movers

In [164]:
movers = identify_movers(after_dropout, preferences)
movers

{0: {1: array([1]), 2: array([1])},
 1: {0: array([57, 36,  4, 77, 15,  0, 39, 18, 47]), 2: array([ 0, 18, 47])},
 2: {0: array([26, 74, 59, 49, 41, 27, 22,  8, 67, 44]),
  1: array([27, 22,  8, 44])}}

In [165]:
def students_who_want_to_move_to_sch(movers):
    schools = movers.keys()
    want_to_move = {t: np.array([], dtype=int) for t in schools}
    for sch1, m in movers.items():
        for sch2, students in m.items():
            want_to_move[sch2] = np.append(want_to_move[sch2], students)
    return want_to_move

In [166]:
want_to_move = students_who_want_to_move_to_sch(movers)
want_to_move

{0: array([57, 36,  4, 77, 15,  0, 39, 18, 47, 26, 74, 59, 49, 41, 27, 22,  8,
        67, 44]),
 1: array([ 1, 27, 22,  8, 44]),
 2: array([ 1,  0, 18, 47])}

In [167]:
def get_students_who_can_move(want_to_move, after_dropout, q, priorities):
    can_move = {}
    for sch in after_dropout.keys():
        empty_seats = q-len(after_dropout[sch])
        student_idx = np.argsort(priorities[want_to_move[sch],sch])[-empty_seats:]
        can_move[sch] = want_to_move[sch][student_idx]
    return can_move

In [168]:
can_move = get_students_who_can_move(want_to_move, after_dropout, q, priorities)
can_move

{0: array([36, 74, 47, 18,  8, 67, 26]),
 1: array([44,  1]),
 2: array([ 1,  0, 18])}

In [169]:
np.concatenate(list(can_move.values()))

array([36, 74, 47, 18,  8, 67, 26, 44,  1,  1,  0, 18])

In [170]:
def update_assignment_with_movers(after_dropout, can_move):
    new_assignment = {}
    movers = set(np.concatenate(list(can_move.values())))
    for sch, students in after_dropout.items():
        staying_students = np.array(list(set(students)-movers))
        new_assignment[sch] = np.append(staying_students, can_move[sch])
    return new_assignment
        

In [171]:
update_assignment_with_movers(after_dropout, can_move)

{0: array([32, 68,  6,  7, 72, 73, 43, 46, 78, 48, 53, 21, 23, 55, 36, 74, 47,
        18,  8, 67, 26]),
 1: array([66,  3,  4, 69, 39, 40, 11, 76, 77, 14, 15, 52, 56, 57, 28, 31, 44,
         1]),
 2: array([ 2, 70, 41, 59, 75, 12, 16, 17, 49, 22, 27, 29, 62, 63,  1,  0, 18])}

In [192]:
np.full(1,2)

array([2])

In [186]:
leaving = np.array(leaving, dtype=int)
leaving

array([42, 10, 45,  5, 34, 24, 33, 60, 19, 38, 50, 79])

In [198]:
def generate_rnd2_preferences(preferences, leaving):
    rnd2_preferences = np.append(preferences, np.full((n,1), 3, dtype=int),1)
    rnd2_preferences[leaving,-1] = rnd2_preferences[leaving,0]
    rnd2_preferences[leaving,0] = 3
    return rnd2_preferences

rnd2_preferences = generate_rnd2_preferences(preferences, leaving)
rnd2_preferences

array([[0, 2, 1, 3],
       [2, 1, 0, 3],
       [2, 0, 1, 3],
       [1, 2, 0, 3],
       [0, 1, 2, 3],
       [3, 0, 2, 1],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [1, 0, 2, 3],
       [1, 0, 2, 3],
       [3, 2, 1, 0],
       [1, 0, 2, 3],
       [2, 0, 1, 3],
       [1, 2, 0, 3],
       [1, 2, 0, 3],
       [0, 1, 2, 3],
       [2, 0, 1, 3],
       [2, 0, 1, 3],
       [0, 2, 1, 3],
       [3, 0, 2, 1],
       [1, 0, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 2, 1, 3],
       [3, 1, 2, 0],
       [2, 0, 1, 3],
       [0, 2, 1, 3],
       [0, 1, 2, 3],
       [1, 0, 2, 3],
       [2, 1, 0, 3],
       [0, 1, 2, 3],
       [1, 0, 2, 3],
       [0, 1, 2, 3],
       [3, 2, 1, 0],
       [3, 1, 2, 0],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [1, 0, 2, 3],
       [3, 2, 1, 0],
       [0, 1, 2, 3],
       [1, 0, 2, 3],
       [0, 2, 1, 3],
       [3, 1, 2, 0],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [3, 0, 2, 1],
       [0, 1, 2, 3],
       [0, 2,

In [195]:
def generate_rnd2_priorities(priorities, after_dropout):
    rnd2_priorities = np.copy(priorities)
    for sch, students in after_dropout.items():
        rnd2_priorities[students,sch] += 1
    return np.append(rnd2_priorities, np.full((n,1),2, dtype=int), axis=1)

rnd2_priorities = generate_rnd2_priorities(priorities, after_dropout)
rnd2_priorities

array([[1.41020133e-01, 1.82989144e+00, 4.56794627e-01, 2.00000000e+00],
       [1.88471943e+00, 5.98510451e-01, 1.41262087e-01, 2.00000000e+00],
       [1.20010758e-02, 1.90847441e-01, 1.64674872e+00, 2.00000000e+00],
       [3.82086767e-02, 1.63563107e+00, 8.55953899e-02, 2.00000000e+00],
       [2.59955395e-02, 1.89709157e+00, 7.00177259e-01, 2.00000000e+00],
       [8.59446791e-01, 6.20906081e-01, 5.35423855e-01, 2.00000000e+00],
       [1.58556344e+00, 9.74921497e-01, 5.32338907e-01, 2.00000000e+00],
       [1.97640364e+00, 3.06240393e-02, 3.42276954e-01, 2.00000000e+00],
       [5.72221621e-01, 3.49784798e-01, 1.74728804e+00, 2.00000000e+00],
       [3.97225828e-01, 3.88375472e-01, 4.03340046e-02, 2.00000000e+00],
       [9.46167374e-01, 9.05682719e-01, 3.59090483e-01, 2.00000000e+00],
       [4.99066134e-01, 1.73984067e+00, 3.01770471e-01, 2.00000000e+00],
       [9.94327151e-01, 8.96855854e-01, 1.67050720e+00, 2.00000000e+00],
       [2.25645872e-01, 4.36706078e-01, 4.11951810e

In [205]:
da2 = DeferredAcceptance(rnd2_preferences, rnd2_priorities, np.array([22,22,22,n]))
r2_assignment_before_removal = da2.da()

In [206]:
r2_assignment_before_removal

{0: array([ 7, 32, 72, 48, 53, 46, 68, 78, 73, 55, 23, 21, 43,  6, 71, 26, 67,
         8, 18, 54, 47, 74]),
 1: array([57, 36,  4, 14, 77, 31, 15, 66, 28, 52, 39, 11, 69, 76, 40,  3, 56,
        44, 30, 64, 51, 13]),
 2: array([59, 16, 49, 41, 63, 27, 29, 22, 70, 62, 12, 17,  2, 75, 58, 37,  0,
        61, 25, 20, 65,  1]),
 3: array([79, 60, 50, 45, 42, 38, 34, 33, 24, 19, 10,  5,  9, 35])}

In [210]:
leaving

array([42, 10, 45,  5, 34, 24, 33, 60, 19, 38, 50, 79])

In [207]:
r2_assignment = {}
for school, students in r2_assignment_before_removal.items():
    r2_assignment[school] = np.array(list(set(students)-set(leaving)))

In [208]:
r2_assignment

{0: array([ 6,  7,  8, 18, 21, 23, 26, 32, 43, 46, 47, 48, 53, 54, 55, 67, 68,
        71, 72, 73, 74, 78]),
 1: array([ 3,  4, 11, 13, 14, 15, 28, 30, 31, 36, 39, 40, 44, 51, 52, 56, 57,
        64, 66, 69, 76, 77]),
 2: array([ 0,  1,  2, 12, 16, 17, 20, 22, 25, 27, 29, 37, 41, 49, 58, 59, 61,
        62, 63, 65, 70, 75]),
 3: array([ 9, 35])}

In [179]:
after_dropout

{0: array([ 7, 32, 72, 48,  1, 53, 46, 68, 78, 73, 55, 23, 21, 43,  6]),
 1: array([57, 36,  4, 14, 77, 31, 15,  0, 66, 28, 52, 39, 11, 69, 76, 40, 18,
        47,  3, 56]),
 2: array([26, 74, 59, 16, 49, 41, 63, 27, 29, 22, 70,  8, 62, 12, 17, 67,  2,
        75, 44])}

In [209]:
for sch in range(3):
    print(len(after_dropout[sch]))
    print(len(r2_assignment[sch]))
    print(set(after_dropout[sch])-set(r2_assignment[sch]))
    print()

15
22
{1}

20
22
{0, 18, 47}

19
22
{67, 8, 74, 44, 26}



In [211]:
qhat_max = int(q/(1-p))
qhat_max

24

In [214]:
def evaluate_assignment(after_dropout):
    overfilled = 0
    empty = 0
    for school, students in after_dropout.items():
        overfilled += len(students) > q
        empty += max(0, q-len(students))
    return np.array([overfilled, empty])
    

In [217]:
utilities = np.tile([6,5,4], (n,1))
mnl = MNL(utilities, num_students=n)
num_iters = 1000

grid_search_results = {}
for q1hat in range(q, qhat_max+1):
    for q2hat in range(q, qhat_max+1):
        for q3hat in range(q, qhat_max+1):
            capacities = np.array([q1hat, q2hat, q3hat])
            results = np.zeros((num_iters,2))
            
            for i in range(num_iters):
                priorities = np.random.uniform(size=(n,3))
                preferences = mnl.sample_preference_ordering()

                da = DeferredAcceptance(preferences,priorities, capacities)
                assignment = da.da()

                after_dropout, leaving = simulate_dropout(assignment)
                results[i,:] = evaluate_assignment(after_dropout)
                
            grid_search_results[(q1hat, q2hat, q3hat)] = results

In [218]:
grid_search_results

{(22,
  22,
  22): array([[ 0.,  6.],
        [ 0., 11.],
        [ 0.,  4.],
        ...,
        [ 0.,  4.],
        [ 0.,  3.],
        [ 0.,  7.]]),
 (22,
  22,
  23): array([[ 0.,  1.],
        [ 0.,  4.],
        [ 0.,  7.],
        ...,
        [ 0.,  8.],
        [ 0.,  6.],
        [ 0., 10.]]),
 (22,
  22,
  24): array([[0., 9.],
        [0., 3.],
        [0., 6.],
        ...,
        [1., 5.],
        [0., 4.],
        [1., 3.]]),
 (22,
  23,
  22): array([[0., 2.],
        [0., 5.],
        [0., 6.],
        ...,
        [0., 9.],
        [0., 4.],
        [0., 9.]]),
 (22,
  23,
  23): array([[ 1.,  2.],
        [ 0.,  5.],
        [ 0.,  9.],
        ...,
        [ 1.,  2.],
        [ 0., 10.],
        [ 1.,  5.]]),
 (22,
  23,
  24): array([[1., 4.],
        [0., 8.],
        [0., 6.],
        ...,
        [0., 6.],
        [0., 7.],
        [1., 7.]]),
 (22,
  24,
  22): array([[0., 7.],
        [1., 5.],
        [0., 3.],
        ...,
        [0., 5.],
        [0., 3.

In [220]:
for k, v in grid_search_results.items():
    print(k, np.sum(v, axis=0)/np.array([num_iters, num_iters*n]))

(22, 22, 22) [0.      0.08325]
(22, 22, 23) [0.09    0.07165]
(22, 22, 24) [0.308     0.0645125]
(22, 23, 22) [0.075   0.07145]
(22, 23, 23) [0.166     0.0617125]
(22, 23, 24) [0.391   0.05385]
(22, 24, 22) [0.29      0.0648875]
(22, 24, 23) [0.396     0.0537625]
(22, 24, 24) [0.607     0.0459125]
(23, 22, 22) [0.103   0.07275]
(23, 22, 23) [0.173    0.061825]
(23, 22, 24) [0.385     0.0551375]
(23, 23, 22) [0.162     0.0623375]
(23, 23, 23) [0.25      0.0540875]
(23, 23, 24) [0.463    0.044125]
(23, 24, 22) [0.364    0.055125]
(23, 24, 23) [0.466  0.0441]
(23, 24, 24) [0.681     0.0347375]
(24, 22, 22) [0.276     0.0646125]
(24, 22, 23) [0.401     0.0559875]
(24, 22, 24) [0.612     0.0471875]
(24, 23, 22) [0.398     0.0547125]
(24, 23, 23) [0.457    0.045025]
(24, 23, 24) [0.698     0.0357375]
(24, 24, 22) [0.573   0.04635]
(24, 24, 23) [0.693  0.0388]
(24, 24, 24) [0.842    0.029625]


In [221]:
x = np.array([1,2,3])
x.append(4)

AttributeError: 'numpy.ndarray' object has no attribute 'append'